In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Subset
from urllib.parse import urlparse
from three_datasets import all_dataset
import random
tqdm.pandas()
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"✅ Seed fixed to {seed}")

  label                                                url
0     2                        https://blog.sockpuppet.us/
1     2                  https://blog.apiki.com/seguranca/
2     1  http://autoecole-lauriston.com/a/T0RVd056QXlNe...
3     1  http://chinpay.site/index.html?hgcFSE@E$Z*DFcG...
4     2  http://www.firstfivenebraska.org/blog/article/...


c:\Users\rrpra\Documents\Github\git_testing\email_and_url_classifier_using_nlp_feature_extraction_sem_5_project\three_datasets.py:70: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df4['label'][df4['label'] == 1] = 0
c:\Users\rrpra\Documents\

✅ Seed fixed to 42


In [3]:
from transformers import AutoTokenizer,  T5EncoderModel
import torch
import numpy as np

model_name = "google/canine-s"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5EncoderModel.from_pretrained(model_name)  
model.eval()
def get_byt5_embedding(url):
    # Tokenize the input
    inputs = tokenizer(url, return_tensors="pt", truncation=True, padding=True, max_length=50)
    
    # Get encoder output (not generate)
    with torch.no_grad():
        outputs = model(**inputs)
        # outputs.last_hidden_state: [batch_size, seq_len, hidden_dim]
         
        # Mean pooling across sequence
        emb = outputs.last_hidden_state.mean(dim=1).reshape(-1)
    return emb.cpu().numpy()

# Example
#url = "http://paypal-login-secure-update.com"
#vector = get_byt5_embedding(url)
#print("Embedding shape:", vector.shape)  # e.g. (512,) or (768,) depending on model size


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framewo

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load once globally (so it doesn't reload every call)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "google/canine-s"  # or "google/canine-c" (larger, slower)

# Load model + tokenizer only once
tokenizer_canine = AutoTokenizer.from_pretrained(MODEL_NAME)
model_canine = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model_canine.eval()

@torch.no_grad()
def get_canine_embedding(url, max_length=50, pooling="mean"):
    """
    Returns a fixed-length CANINE embedding for a single URL string.

    Args:
        url (str): URL text to encode.
        max_length (int): Maximum length of character sequence (truncate/pad to this).
        pooling (str): 'mean' or 'max' for pooling token embeddings into one vector.

    Returns:
        np.ndarray: 1D embedding vector (shape [hidden_dim], e.g. [768]).
    """
    import numpy as np

    # Tokenize
    encoded = tokenizer_canine(
        url,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt"
    ).to(DEVICE)

    # Forward pass
    outputs = model_canine(**encoded)
    hidden = outputs.last_hidden_state  # [1, seq_len, hidden_dim]
    mask = encoded["attention_mask"].unsqueeze(-1).type_as(hidden)  # [1, seq_len, 1]

    # Pooling
    if pooling == "mean":
        summed = (hidden * mask).sum(dim=1)
        denom = mask.sum(dim=1).clamp(min=1e-9)
        pooled = summed / denom  # [1, hidden_dim]
    elif pooling == "max":
        hidden = hidden.masked_fill(mask == 0, -1e9)
        pooled = hidden.max(dim=1).values  # [1, hidden_dim]
    else:
        raise ValueError("Pooling must be 'mean' or 'max'")

    return pooled.squeeze(0).cpu().numpy()  # (hidden_dim,)


In [2]:
# ============================================================
# Character Encoding Setup
# ============================================================

# Allowed printable ASCII chars
ascii_chars = [chr(i) for i in range(32, 127)]

# Special control tokens
special_tokens = [
    '<PAD>', '<UNK>',
]

# Build vocab and mapping
vocab = special_tokens + ascii_chars
char2idx = {ch: i for i, ch in enumerate(vocab)}



def encode(text, max_len=100):
    indices = torch.full((max_len,), char2idx["<PAD>"], dtype=torch.long)
    text = text.lower()[:max_len]
    for i, c in enumerate(text):
        indices[i] = char2idx.get(c, char2idx["<UNK>"])

    return indices





encoded_data = {}
frac = 1
gen = all_dataset()

x=0
next(gen)
next(gen)
for name, splits in  gen:
    encoded_data[name] = {}
    print(f"\n🔠 Encoding structured URLs for {name}...")

    for split_name, df in zip(['train', 'valid', 'test'], splits):
        df = df.sample(frac=frac, random_state=42)
        df["encode"] = df["url"].progress_apply(encode)
        encoded_data[name][split_name] = df
        print(f"  ✅ {split_name}: Encoded {len(df)} URLs")

    x+=1
    if x > 0:
        next(gen)

print("\n✅ All datasets encoded with proper start/end markers and padding!")


🔠 Encoding structured URLs for Dataset 3 (kmack/Phishing_urls)...


100%|██████████| 528097/528097 [01:19<00:00, 6638.43it/s]


  ✅ train: Encoded 528097 URLs


100%|██████████| 66012/66012 [00:09<00:00, 6694.53it/s]


  ✅ valid: Encoded 66012 URLs


100%|██████████| 66013/66013 [00:09<00:00, 6666.52it/s]

  ✅ test: Encoded 66013 URLs

✅ All datasets encoded with proper start/end markers and padding!


In [3]:
from torch.utils.data import TensorDataset, DataLoader
# ============================================================
# Convert to TensorDataset and DataLoader
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32

dataloader_dict = {}

def make_tensor_dataset(df):
    url_tensor = torch.stack(list(df["encode"]))
    labels_tensor = torch.tensor(df["label"].values, dtype=torch.long)
    return TensorDataset(url_tensor, labels_tensor)

for name, splits in encoded_data.items():
    dataloader_dict[name] = {}
    print(f"\n📦 Creating DataLoaders for {name}...")
    
    train_set = make_tensor_dataset(splits["train"])
    val_set = make_tensor_dataset(splits["valid"])
    test_set = make_tensor_dataset(splits["test"])
    
    dataloader_dict[name]["train_loader"] = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    dataloader_dict[name]["val_loader"] = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    dataloader_dict[name]["test_loader"] = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    
    print(f"✅ DataLoaders ready for {name} (Train/Val/Test)")

print("\n🚀 All DataLoaders are ready in `dataloader_dict`!")

# Example Access:
# dataloader_dict["dataset1"]["train_loader"]



📦 Creating DataLoaders for Dataset 3 (kmack/Phishing_urls)...
✅ DataLoaders ready for Dataset 3 (kmack/Phishing_urls) (Train/Val/Test)

🚀 All DataLoaders are ready in `dataloader_dict`!


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# =====================================================
# 🔹 TinyByT5 Encoder (Reduced depth for short sequences)
# =====================================================
class TinyByT5Encoder(nn.Module):
    def __init__(self,
                 vocab_size=256,
                 d_model=128,
                 num_layers=2,
                 num_heads=2,
                 dim_ff=256,
                 max_len=100,
                 n_out=128,
                 dropout=0.1):
        super().__init__()

        # 🔹 Byte embedding layer (0–255)
        self.embedding = nn.Embedding(vocab_size, d_model)

        # 🔹 Positional embeddings
        self.pos_embedding = nn.Embedding(max_len, d_model)

        # 🔹 Transformer encoder layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=dim_ff,
            dropout=dropout,
            activation="gelu",
            batch_first=True,
            norm_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # 🔹 Final normalization
        self.final_norm = nn.LayerNorm(d_model)
        self.projection = nn.Linear(in_features=d_model, out_features=n_out)

    def forward(self, x):
        """
        x: (batch, seq_len) — byte indices [0–255]
        """
        #batch_size, seq_len = x.size()
        #device = x.device

        #positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        #x = self.embedding(x) + self.pos_embedding(positions)

        #x = self.encoder(x)
        x = self.embedding(x)
        x = self.projection(self.final_norm(x))

        return x  # (B, L, d_model)

In [5]:
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=8):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, C, L)
        w = x.mean(dim=2)                          # Global Average Pooling -> (B, C)
        w = F.relu(self.fc1(w))
        w = self.sigmoid(self.fc2(w))
        w = w.unsqueeze(2)                         # (B, C, 1)
        return x * w                               # scale features

In [6]:
# 🔹 Residual Depthwise-Separable Multi-Kernel Block
class ResidualConvBlockDW(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_sizes=[3, 5, 7], reduction=16):
        super().__init__()
        self.branches = nn.ModuleList()

        mid_ch = max(in_ch // 16, 8)  # reduce dimension before heavy convs

        for k in kernel_sizes:
            branch = nn.Sequential(
                # (B) Reduce channels first
                nn.Conv1d(in_ch, mid_ch, kernel_size=1, bias=False),
                #nn.BatchNorm1d(mid_ch),
                nn.ReLU(inplace=True),
                nn.Dropout1d(0.25),

                # (A) Depthwise conv
                nn.Conv1d(mid_ch, mid_ch, kernel_size=k, padding=k // 2, groups=mid_ch, bias=False),
                #nn.BatchNorm1d(mid_ch),
                nn.ReLU(inplace=True),
                nn.Dropout1d(0.25),

                # Pointwise to expand to out_ch
                nn.Conv1d(mid_ch, out_ch, kernel_size=1, bias=False),
                #nn.BatchNorm1d(out_ch),
                nn.ReLU(inplace=True)
            )
            self.branches.append(branch)

        # Combine all kernel branches
        self.merge_conv = nn.Conv1d(out_ch * len(kernel_sizes), out_ch, kernel_size=1, bias=False)
        #self.merge_bn = nn.BatchNorm1d(out_ch)

        self.se = SEBlock(out_ch, reduction)
        self.shortcut = nn.Conv1d(in_ch, out_ch, kernel_size=1) if in_ch != out_ch else nn.Identity()

    def forward(self, x):
        # Parallel multi-kernel branches
        out = [branch(x) for branch in self.branches]
        out = torch.cat(out, dim=1)

        out = self.merge_conv(out)
        #out = self.merge_bn(out)
        out = self.se(out)
        out += self.shortcut(x)
        return F.relu(out)

    


In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class URLBinaryCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, maxlen=100):
        super(URLBinaryCNN, self).__init__()
        self.maxlen = maxlen

        self.transformer_layer = nn.ModuleDict({
            "transformer": TinyByT5Encoder(vocab_size=vocab_size, max_len=maxlen, d_model=512, n_out=embed_dim)
        })

        # 2️⃣ Shared Layer (Global)
        self.shared_layer = nn.ModuleDict({
            "conv": ResidualConvBlockDW(embed_dim, 128),
            "bilstm": nn.LSTM(input_size=128, hidden_size=128, batch_first=True, bidirectional=True),
            "layer_norm": nn.LayerNorm(128 * 2),
            "fc1": nn.Linear(128 * 2, 128),
            "relu1": nn.ReLU(),
            "dropout1": nn.Dropout(0.5),
            "fc2": nn.Linear(128, 64),
            "relu2": nn.ReLU(),
            "dropout2": nn.Dropout(0.5),
        })

        # 3️⃣ Personalization Layer (Local)
        self.personal_layer = nn.ModuleDict({
            "head": nn.Linear(64, 1)
        })

    def forward(self, x):
        # Transformer
        x = self.transformer_layer["transformer"](x)
        x = x.permute(0, 2, 1)
        # Shared layers
        x = self.shared_layer["conv"](x)
        x = x.permute(0, 2, 1)
        x, _ = self.shared_layer["bilstm"](x)
        x = self.shared_layer["layer_norm"](x)
        x = x[:, -1, :]
        x = self.shared_layer["dropout2"](self.shared_layer["relu2"](self.shared_layer["fc2"](
            self.shared_layer["dropout1"](self.shared_layer["relu1"](self.shared_layer["fc1"](x)))
        )))
        # Personalization head
        x = self.personal_layer["head"](x)
        return torch.sigmoid(x)
    
    def extract_features(self, x):
        """Return deep features before final FC layers."""
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        x = self.conv_layer1(x)
        x = x.permute(0, 2, 1)
        x, _ = self.bilstm(x)
        x = x[:, -1, :]          # shape: [batch, 32]
        x = self.flatten(x) 
        x = self.dropout(self.relu(self.fc1(x)))
        return x



In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import sys

class Train:
    def __init__(self, 
                 model, 
                 criterion, 
                 transformer_optimizer=None,
                 main_optimizer = None, 
                 scheduler_t=None,
                 scheduler_c=None,
                 train_loader=None, 
                 val_loader=None,   
                 device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
        
        """
        optimizer_groups: dict with keys like {"transformer": optimizer1, "cnn": optimizer2}
        schedulers: dict with keys matching optimizer_groups (optional)
        """
        self.model = model
        self.criterion = criterion

        self.transformer_params = []
        self.cnn_params = []
        for name, param in model.transformer_layer.named_parameters():
            self.transformer_params.append(param)
        for name, param in model.shared_layer.named_parameters():
            self.cnn_params.append(param)
        for name, param in model.personal_layer.named_parameters():
            self.cnn_params.append(param)



        self.transformer_optimizer = optim.NAdam(self.transformer_params, lr=1e-4) if transformer_optimizer is None  else transformer_optimizer
        self.main_optimizer = optim.NAdam(self.cnn_params, lr=1e-3) if main_optimizer is None else main_optimizer
        self.scheduler_t = optim.lr_scheduler.ReduceLROnPlateau(self.transformer_optimizer, mode='min', factor=0.5, patience=2) if scheduler_t is None  else scheduler_t
        self.scheduler_c = optim.lr_scheduler.ReduceLROnPlateau(self.main_optimizer, mode='min', factor=0.5, patience=2) if scheduler_c is None  else scheduler_c
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.train_losses, self.val_losses = [], []
        self.train_accs, self.val_accs = [], []


    def freeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = False

    def unfreeze_module(self, module):
        for param in module.parameters():
            param.requires_grad = True


    def train(self, epochs_list=[3,3,4], early_stopping=True, frac=1.0, val_frac=1.0, alt_cycle = 2,start=0,  log=0):
        for phase, epochs in enumerate(epochs_list):
            for epoch in range(epochs):
                phase=3
                if phase == 0:
                    # 🧠 Train Transformer — freeze CNN/LSTM layers
                    for name, module in self.model.named_children():
                        if "transformer" not in name:
                            self.freeze_module(module)
                        else:
                            self.unfreeze_module(module)
                    active_optims = [self.transformer_optimizer]
                    active_scheds = [self.scheduler_t]
                    phase_name = "Transformer"
                elif phase == 1:
                    # 🎯 Train CNN/LSTM/FC — freeze Transformer

                    for name, module in self.model.named_children():
                        if "transformer" in name:
                            self.freeze_module(module)
                        else:
                            self.unfreeze_module(module)
                    active_optims = [self.main_optimizer]
                    active_scheds = [self.scheduler_c]
                    phase_name = "CNN"
                else:

                    for name, module in self.model.named_children():
                        self.unfreeze_module(module)
                    active_optims = [self.transformer_optimizer, self.main_optimizer]
                    active_scheds = [self.scheduler_t, self.scheduler_c]
                    phase_name = "CNN + transformer"



                self.model.train()
                train_loss, correct_train, total_train = 0, 0, 0
                max_batches = int(len(self.train_loader) * (frac+start)) +1 
                start_batch = int(len(self.train_loader) * start)
                
                for batch_idx, (batch_x, batch_y) in enumerate(self.train_loader):
                    if batch_idx < start_batch:
                        continue
                    elif batch_idx >= max_batches:
                        break
                    
                    
                    batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                    
                    for opt in active_optims:
                        opt.zero_grad()

                    outputs = self.model(batch_x)
                    loss = self.criterion(outputs, batch_y)
                    loss.backward()

                    for opt in active_optims:
                        opt.step()




                    # === Metrics ===
                    batch_loss = loss.item()
                    preds = (outputs >= 0.5).float()
                    batch_acc = (preds == batch_y).float().mean().item()

                    if log >= 1 and (batch_idx + 1) % (20/log) == 0:
                        print(f"\rEpoch {epoch+1}/{epochs}: Training {phase_name} | "
                            f"Batch {batch_idx+1}/{max_batches} | "
                            f"Loss: {batch_loss:.4f}, Acc: {batch_acc:.4f}", end='')


                if log >= 2:
                    print(f'\r total training batch size {max_batches-start_batch}'.ljust(100), end='')
                    with torch.no_grad():
                        for batch_idx, (batch_x, batch_y) in enumerate(self.train_loader):
                            if batch_idx < start_batch:
                                continue
                            elif batch_idx >= max_batches:
                                break
                            batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)

                            outputs = self.model(batch_x)
                            loss = self.criterion(outputs, batch_y)

                            batch_loss = loss.item()
                            preds = (outputs >= 0.5).float()
                            batch_acc = (preds == batch_y).float().mean().item()

                            train_loss += batch_loss * batch_x.size(0)
                            correct_train += (preds == batch_y).sum().item()
                            total_train += batch_x.size(0)

                        avg_train_loss = train_loss / total_train
                        train_acc = correct_train / total_train
                        self.train_losses.append(avg_train_loss)
                        self.train_accs.append(train_acc)


                    # === Validation ===
                    if self.val_loader is not None:
                        avg_val_loss, val_acc = self.evaluate(val_frac)
                        self.val_losses.append(avg_val_loss)
                        self.val_accs.append(val_acc)

                        print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                            f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
                            f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")
                    else:
                        print(f"\rEpoch {epoch+1}/{epochs} Training {phase_name}| "
                            f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f}")


                for sched in active_scheds:
                    sched.step(avg_val_loss)



    def evaluate(self, frac=1.0, start=0, log=2):
        self.model.eval()
        val_loss, correct_val, total_val = 0, 0, 0
        max_batches = max(int(len(self.val_loader) * (frac+start)), 0)+1
        start_batch = int(len(self.val_loader) * start)
        if log > 2:
            print('\rstarting from batch', start_batch, 'ending to batch', max_batches, f'total validation batch size {max_batches}')
        with torch.no_grad():
            for batch_idx, (batch_x, batch_y) in enumerate(self.val_loader):
                if batch_idx < start_batch:
                    continue
                elif batch_idx >= max_batches:
                    break
                batch_x, batch_y = batch_x.to(self.device, non_blocking=True), batch_y.to(self.device, non_blocking=True).float().unsqueeze(1)
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y)
                avg_batch_loss = loss.item()
                val_loss += avg_batch_loss * batch_x.size(0)
                preds = (outputs >= 0.5).float()
                correct_val += (preds == batch_y).sum().item()
                total_val += batch_x.size(0)
        avg_val_loss = val_loss / total_val
        val_acc = correct_val / total_val
        return avg_val_loss, val_acc
    



In [ ]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from lion_pytorch import Lion
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# 🔧 Training Config
# ============================================================
num_epochs = [2,2,1]
lr = 0.01

# Store all dataset metrics
all_results = {}

# ============================================================
# 🔁 Training Loop for Each Dataset
# ============================================================
nn_model = {}
for dataset_name, loaders in dataloader_dict.items():
    print("\n" + "="*70)
    print(f"🚀 Training model on {dataset_name.upper()} dataset")
    print("="*70)
    for frac in  [1.0]:
        print(f"🧩 Using {frac*100:.0f}% of training data".center(50, '_'))
        train_loader = loaders["train_loader"]
        val_loader = loaders["val_loader"]

        # Initialize model, loss, optimizer
        nn_model[dataset_name] = URLBinaryCNN(vocab_size=len(vocab)).to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(nn_model[dataset_name].parameters(), lr=lr, weight_decay=lr/10)
        trainer = Train(nn_model[dataset_name], criterion, train_loader=train_loader, val_loader=val_loader)
        # Lists to track performance
        trainer.train(num_epochs,frac=frac,val_frac=frac, log=2)

    

    # Store all results for this dataset
    all_results[dataset_name] = {
        "train_losses": trainer.train_losses,
        "val_losses": trainer.val_losses,
        "train_accs": trainer.train_accs,
        "val_accs": trainer.val_accs,
        "final_val_acc": trainer.val_accs[-1],
        "final_val_loss": trainer.val_losses[-1]
    }

print("\n✅ All datasets trained successfully!")

# ============================================================
# 📊 Summary of All Results
# ============================================================
print("\n" + "="*70)
print("📈 Final Validation Accuracy Summary")
print("="*70)
for name, res in all_results.items():
    print(f"{name:<20} | Val Acc: {res['final_val_acc']:.4f} | Val Loss: {res['final_val_loss']:.4f}")



🚀 Training model on DATASET 2 (NDARVIND/PHIUSIIL-PHISHING) dataset
__________🧩 Using 100% of training data___________


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Epoch 1/2 Training CNN + transformer| Train Loss: 0.0158, Train Acc: 0.9975 | Val Loss: 0.0154, Val Acc: 0.9972
Epoch 2/2 Training CNN + transformer| Train Loss: 0.0142, Train Acc: 0.9979 | Val Loss: 0.0144, Val Acc: 0.9976
Epoch 1/2 Training CNN + transformer| Train Loss: 0.0143, Train Acc: 0.9980 | Val Loss: 0.0147, Val Acc: 0.9975
Epoch 2/2 Training CNN + transformer| Train Loss: 0.0151, Train Acc: 0.9979 | Val Loss: 0.0151, Val Acc: 0.9975
Epoch 1/1 Training CNN + transformer| Train Loss: 0.0171, Train Acc: 0.9975 | Val Loss: 0.0164, Val Acc: 0.9973

🚀 Training model on DATASET 3 (KMACK/PHISHING_URLS) dataset
__________🧩 Using 100% of training data___________
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2483, Train Acc: 0.9014 | Val Loss: 0.2464, Val Acc: 0.9010
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2274, Train Acc: 0.9068 | Val Loss: 0.2317, Val Acc: 0.9050
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2141, Train Acc: 0.9145 | Val Loss: 0.2169, Val Acc:

In [22]:
torch.save(model.state_dict(), "url_cnn_lstm.pth")


In [23]:
#xgbost model + cnn based dl model
def extract_features(self, x):
        """Return deep features before final FC layers."""
        x = self.transformer(x)
        x = x.permute(0, 2, 1)
        x3 = F.relu(self.conv1_3x3(x))
        x5 = F.relu(self.conv1_5x5(x))
        x7 = F.relu(self.conv1_7(x))
        x = torch.cat([x3, x5, x7], dim=1)
        x = F.relu(self.conv1_1x1(x))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]          # shape: [batch, 32]
        return x

In [24]:
import numpy as np
from tqdm import tqdm
import torch

model.eval()
features, labels = [], []

# 🔹 Extract CNN features for training set
with torch.no_grad():
    for x_batch, y_batch in tqdm(train_loader, desc="🔍 Extracting Train Features", unit="batch"):
        x_batch = x_batch.to(device, non_blocking=True)
        feats = nn_model[dataset_name].extract_features(x_batch)
        features.append(feats.cpu().numpy())
        labels.append(y_batch.cpu().numpy())

X_train = np.concatenate(features, axis=0)
y_train = np.concatenate(labels, axis=0)

# Free memory before val extraction
del features, labels, x_batch, y_batch, feats
torch.cuda.empty_cache()

# 🔹 Extract CNN features for validation set
features, labels = [], []
with torch.no_grad():
    for x_batch, y_batch in tqdm(val_loader, desc="🔍 Extracting Val Features", unit="batch"):
        x_batch = x_batch.to(device, non_blocking=True)
        feats = nn_model[dataset_name].extract_features(x_batch)
        features.append(feats.cpu().numpy())
        labels.append(y_batch.cpu().numpy())

X_val = np.concatenate(features, axis=0)
y_val = np.concatenate(labels, axis=0)


🔍 Extracting Train Features:   0%|          | 0/16504 [00:00<?, ?batch/s]

🔍 Extracting Val Features: 100%|██████████| 2063/2063 [00:11<00:00, 174.84batch/s]


In [25]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.01,
    eval_metric="logloss",
    max_depth = 10,
    random_state=42
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=True
)


[0]	validation_0-logloss:0.68574
[1]	validation_0-logloss:0.67847
[2]	validation_0-logloss:0.67135
[3]	validation_0-logloss:0.66437
[4]	validation_0-logloss:0.65753
[5]	validation_0-logloss:0.65081
[6]	validation_0-logloss:0.64423
[7]	validation_0-logloss:0.63777
[8]	validation_0-logloss:0.63142
[9]	validation_0-logloss:0.62520
[10]	validation_0-logloss:0.61910
[11]	validation_0-logloss:0.61310
[12]	validation_0-logloss:0.60722
[13]	validation_0-logloss:0.60143
[14]	validation_0-logloss:0.59576
[15]	validation_0-logloss:0.59019
[16]	validation_0-logloss:0.58471
[17]	validation_0-logloss:0.57933
[18]	validation_0-logloss:0.57405
[19]	validation_0-logloss:0.56885
[20]	validation_0-logloss:0.56375
[21]	validation_0-logloss:0.55873
[22]	validation_0-logloss:0.55380
[23]	validation_0-logloss:0.54895
[24]	validation_0-logloss:0.54419
[25]	validation_0-logloss:0.53951
[26]	validation_0-logloss:0.53490
[27]	validation_0-logloss:0.53037
[28]	validation_0-logloss:0.52592
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, ...)

In [26]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

y_pred_prob = xgb_model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1:", f1_score(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_pred_prob))


Accuracy: 0.9130764103496334
Precision: 0.8966842304445285
Recall: 0.9336485708223455
F1: 0.914793145436726
ROC AUC: 0.913085755384826


In [27]:
#random forest
from sklearn.ensemble import RandomForestClassifier
rfc_model = RandomForestClassifier(
    n_estimators=100,   
    max_depth=30,     
    random_state=42
)



rfc_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=30, random_state=42)

In [28]:
y_pred_prob = rfc_model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1:", f1_score(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_pred_prob))

Accuracy: 0.9119705508089438
Precision: 0.8970665575877987
Recall: 0.9306477524173259
F1: 0.9135486558459913
ROC AUC: 0.911979035046978


In [29]:
#adding xgboost with cnn with handcrafted features
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import xgboost as xgb
import pandas as pd




In [30]:
import pandas as pd
import re
import math
from urllib.parse import urlparse
import tldextract
from rapidfuzz import process, fuzz



import pickle
f= open('tld_encoding_serise_dataset_1.bin','rb')
tld_stats = pickle.load(file=f)
f.close()
print(type(tld_stats))

import kagglehub
import os
# Download latest version
folder_path = kagglehub.dataset_download("cheedcheed/top1m")

print("Path to dataset files:", folder_path)
csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]

if not csv_files:
    raise FileNotFoundError("No CSV files found in the folder!")

# read the first CSV file
file_path = os.path.join(folder_path, csv_files[0])
alexa_top_1m_domain = pd.read_csv(file_path,header=None,names=['rank', 'domain'])
alexa_domains_set = set(alexa_top_1m_domain['domain'].apply(str.lower))

# --- Helper function: Shannon entropy ---
def safe_parse(url: str):
    """Safely parse URLs, adding http:// if missing and handling bad IPv6 parts."""
    if not isinstance(url, str) or not url.strip():
        return urlparse("http://")  

    # Ensure scheme exists
    if not re.match(r'^[a-zA-Z]+://', url):
        url = 'http://' + url

    # Clean invalid brackets that trigger IPv6 errors
    url = re.sub(r'\[.*?\]', '', url)

    try:
        return urlparse(url)
    except ValueError:
        # fallback: strip more aggressively if still malformed
        url = re.sub(r'[^a-zA-Z0-9:/._\-?&=]', '', url)
        return urlparse(url)
def calculate_entropy(string):
    """Measures randomness of characters in the URL."""
    if not string:
        return 0
    freq = {char: string.count(char) for char in set(string)}
    entropy = -sum((count / len(string)) * math.log2(count / len(string)) for count in freq.values())
    return entropy

# --- Main feature extraction function ---
def extract_handcrafted_features(url):
    features = {}
    if not re.match(r'^[hH]+[tT]+[tT]+[pP]+[sS]+://', url):
        url = 'http://' + url
    parsed = safe_parse(url)
    
    # 1️⃣ Basic structural features
    features['url_length'] = len(url)
    features['hostname_length'] = len(parsed.netloc)
    features['path_length'] = len(parsed.path)
    features['num_dots'] = url.count('.')
    features['num_hyphens'] = url.count('-')
    features['num_digits'] = sum(c.isdigit() for c in url)
    features['num_letters'] = sum(c.isalpha() for c in url)
    features['num_params'] = url.count('?')
    features['num_equals'] = url.count('=')
    features['num_slashes'] = url.count('/')
    features['num_at'] = url.count('@')

    # 2️⃣ Lexical / composition cues
    features['has_https'] = 1 if url.lower().startswith('https') else 0
    features['has_ip'] = 1 if re.search(r'(\d{1,3}\.){3}\d{1,3}', parsed.netloc) else 0
    features['has_subdomain'] = 1 if parsed.netloc.count('.') > 1 else 0
    features['has_suspicious_words'] = 1 if re.search(r'(login|secure|verify|update|free|bank|click)', url.lower()) else 0

    # 3️⃣ Domain / TLD features
    extracted = tldextract.extract(url)
    main_domain = f"{extracted.domain}.{extracted.suffix}"
    if ':' in main_domain:  # remove port
        main_domain = main_domain.split(':')[0]
    features['domain_length'] = len(main_domain)
    features['in_alexa_top1m'] = 1 if main_domain in alexa_domains_set else 0
    '''
    if features['in_alexa_top1m'] == 0 and main_domain:  # only check if domain not in top1M
        # find closest match in Alexa domains
        best_match, score, _ = process.extractOne(main_domain, alexa_domains_set, scorer=fuzz.ratio)
        features['closest_alexa_domain'] = best_match
        features['closest_alexa_score'] = score  # 0-100
    else:
        features['closest_alexa_score'] = 1000  # high score to show that it is original url
    '''
    ext = tldextract.extract(url)
    tld = ext.suffix    # "com", "co.uk", "org"
    features['tld'] = tld if tld else 'unknown'
    features['tld_phish_ratio'] = tld_stats['phish_ratio'].get(features['tld'], 0.5)
    features['tld_total_frequency'] = tld_stats['total'].get(features['tld'], 1)

    # 4️⃣ Ratios
    features['digit_ratio'] = features['num_digits'] / (features['url_length'] + 1e-5)
    features['special_char_ratio'] = (features['num_hyphens'] + features['num_dots'] + features['num_slashes']) / (features['url_length'] + 1e-5)

    # 5️⃣ Entropy (measures randomness / obfuscation)
    features['url_entropy'] = calculate_entropy(url)

    # 6️⃣ Misplacement indicators
    # '@' symbol used to hide real domain (like "http://evil.com@legit.com")
    features['at_in_domain'] = 1 if '@' in parsed.netloc else 0
    
    # Double slashes '//' appearing after path (used to trick users)
    features['double_slash_in_path'] = 1 if re.search(r'/.+//', parsed.path) else 0

    return features

<class 'pandas.core.frame.DataFrame'>
Path to dataset files: C:\Users\rrpra\.cache\kagglehub\datasets\cheedcheed\top1m\versions\1


In [31]:
from tqdm import tqdm
tqdm.pandas()
for i in encoded_data:

    for split in encoded_data[i]:
        print("feature extracting", i, split)
        encoded_data[i][split] = encoded_data[i][split].assign(**encoded_data[i][split].url.progress_apply(lambda url : pd.Series(extract_handcrafted_features(url))))

feature extracting Dataset 2 (ndarvind/phiusiil-phishing) train


100%|██████████| 188296/188296 [00:37<00:00, 5035.99it/s]


feature extracting Dataset 2 (ndarvind/phiusiil-phishing) valid


100%|██████████| 23537/23537 [00:04<00:00, 4863.36it/s]


feature extracting Dataset 2 (ndarvind/phiusiil-phishing) test


...

100%|██████████| 528097/528097 [01:51<00:00, 4733.57it/s]


feature extracting Dataset 3 (kmack/Phishing_urls) valid


100%|██████████| 66012/66012 [00:13<00:00, 5000.34it/s]


feature extracting Dataset 3 (kmack/Phishing_urls) test


100%|██████████| 66013/66013 [00:13<00:00, 4794.54it/s]


In [32]:
# Split features and labels
independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependet_features  = 'label'
model_dict = {}
for i in encoded_data:
    print(f"train for {i}")
    X_train = encoded_data[i]['train'][independent_features]
    y_train = encoded_data[i]['train'][dependet_features]
    X_test = encoded_data[i]['valid'][independent_features]
    y_test = encoded_data[i]['valid'][dependet_features]

    # Create XGBoost classifier
    model_dict[i] = xgb.XGBClassifier(
        n_estimators=100,      # number of boosting rounds
        learning_rate=0.01,     # step size shrinkage
        max_depth=30,           # tree depth
        eval_metric='logloss', # evaluation metric
        use_label_encoder=False
    )

    # Train the model
    model_dict[i].fit(X_train, y_train)
    # Predict
    y_pred = model_dict[i] .predict(X_test)

    # Evaluate
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:17:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


train for Dataset 2 (ndarvind/phiusiil-phishing)
Accuracy: 0.9973233632153631

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00     10052
           1       1.00      1.00      1.00     13485

    accuracy                           1.00     23537
   macro avg       1.00      1.00      1.00     23537
weighted avg       1.00      1.00      1.00     23537

train for Dataset 3 (kmack/Phishing_urls)


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:199: UserWarning: [02:17:05] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.8882778888686905

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.89     33021
           1       0.88      0.90      0.89     32991

    accuracy                           0.89     66012
   macro avg       0.89      0.89      0.89     66012
weighted avg       0.89      0.89      0.89     66012



In [33]:
#logistic_regression(nn+xgboost)
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependent_features  = 'label'
logistic_model = {}
for name in encoded_data:
    #print(model_dict[name])
    #print(nn_model[name])
    nn_model[name].eval()
    with torch.no_grad():
        model = nn_model[name]
        #print(model)
        x_ = encoded_data[name]['train']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_train = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_train.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_train = torch.cat(nn_preds_train, dim=0).cpu()

    # XGBoost predictions
    xgb_preds_train = model_dict[name].predict_proba(encoded_data[name]['train'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X = np.column_stack((nn_preds_train, xgb_preds_train))
    meta_y = encoded_data[name]['train'][dependent_features]


    meta_model= LogisticRegressionCV(
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    penalty="l2",
    scoring="roc_auc",
    solver="lbfgs",
    Cs=10,
    max_iter=1000,
    n_jobs=-1
)
    meta_model.fit(meta_X, meta_y)


    with torch.no_grad():
        x_ = encoded_data[name]['valid']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_val = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_val.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_val = torch.cat(nn_preds_val, dim=0).cpu()
    # XGBoost predictions
    xgb_preds_val = model_dict[name].predict_proba(encoded_data[name]['valid'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X_val = np.column_stack((nn_preds_val, xgb_preds_val))
    meta_y_val = encoded_data[name]['valid'][dependent_features]


    y_pred_train = meta_model.predict(meta_X)
    y_pred_prob_train = meta_model.predict_proba(meta_X)[:, 1]

    print("\n📘 TRAIN METRICS")
    print("Accuracy :", accuracy_score(meta_y, y_pred_train))
    print("Precision:", precision_score(meta_y, y_pred_train))
    print("Recall   :", recall_score(meta_y, y_pred_train))
    print("F1-score :", f1_score(meta_y, y_pred_train))
    print("ROC AUC  :", roc_auc_score(meta_y, y_pred_prob_train))

    # ===============================
    # 🔹 Validation Metrics
    # ===============================
    y_pred_val = meta_model.predict(meta_X_val)
    y_pred_prob_val = meta_model.predict_proba(meta_X_val)[:, 1]

    print("\n📗 VALIDATION METRICS")
    print("Accuracy :", accuracy_score(meta_y_val, y_pred_val))
    print("Precision:", precision_score(meta_y_val, y_pred_val))
    print("Recall   :", recall_score(meta_y_val, y_pred_val))
    print("F1-score :", f1_score(meta_y_val, y_pred_val))
    print("ROC AUC  :", roc_auc_score(meta_y_val, y_pred_prob_val))



Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 368/368 [00:15
Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 46/46 [00:01<0



📘 TRAIN METRICS
Accuracy : 0.9982208862641798
Precision: 0.9971707795149644
Recall   : 0.999731182795699
F1-score : 0.9984493396964409
ROC AUC  : 0.9991011402829868

📗 VALIDATION METRICS
Accuracy : 0.9977482261970515
Precision: 0.9965986394557823
Recall   : 0.9994809047089358
F1-score : 0.9980376911399904
ROC AUC  : 0.9991227338271097


Predicting NN for Dataset 3 (kmack/Phishing_urls): 100%|█| 1032/1032 [00:45<00:0
Predicting NN for Dataset 3 (kmack/Phishing_urls): 100%|█| 129/129 [00:04<00:00,



📘 TRAIN METRICS
Accuracy : 0.9406359059036503
Precision: 0.9348845558372787
Recall   : 0.9471901427662089
F1-score : 0.9409971204336288
ROC AUC  : 0.9881483052684124

📗 VALIDATION METRICS
Accuracy : 0.9027904017451372
Precision: 0.8939456831119544
Recall   : 0.9139159164620654
F1-score : 0.9038205009067882
ROC AUC  : 0.9689706976484784


In [34]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependent_features  = 'label'
for name in encoded_data:
    #print(model_dict[name])
    #print(nn_model[name])
    nn_model[name].eval()
    with torch.no_grad():
        model = nn_model[name]
        #print(model)
        x_ = encoded_data[name]['train']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_train = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_train.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_train = torch.cat(nn_preds_train, dim=0).cpu()

    # XGBoost predictions
    xgb_preds_train = model_dict[name].predict_proba(encoded_data[name]['train'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X = np.column_stack((nn_preds_train, xgb_preds_train))
    meta_y = encoded_data[name]['train'][dependent_features]


    params = {
    "hidden_layer_sizes": [(8,), (16,), (24,)],
    "alpha": [1e-3, 1e-4, 1e-5],  # L2 regularization
    "learning_rate_init": [1e-3],
    "early_stopping": [True],
    "max_iter": [500],
    }

    mlp = MLPClassifier(random_state=42)
    meta_model = GridSearchCV(mlp, params, cv=StratifiedKFold(3), scoring="accuracy", n_jobs=-1, verbose=3)
    meta_model.fit(meta_X, meta_y)


    with torch.no_grad():
        x_ = encoded_data[name]['valid']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_val = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_val.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_val = torch.cat(nn_preds_val, dim=0).cpu()
    # XGBoost predictions
    xgb_preds_val = model_dict[name].predict_proba(encoded_data[name]['valid'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X_val = np.column_stack((nn_preds_val, xgb_preds_val))
    meta_y_val = encoded_data[name]['valid'][dependent_features]


    y_pred_train = meta_model.predict(meta_X)
    y_pred_prob_train = meta_model.predict_proba(meta_X)[:, 1]

    print("\n📘 TRAIN METRICS")
    print("Accuracy :", accuracy_score(meta_y, y_pred_train))
    print("Precision:", precision_score(meta_y, y_pred_train))
    print("Recall   :", recall_score(meta_y, y_pred_train))
    print("F1-score :", f1_score(meta_y, y_pred_train))
    print("ROC AUC  :", roc_auc_score(meta_y, y_pred_prob_train))

    # ===============================
    # 🔹 Validation Metrics
    # ===============================
    y_pred_val = meta_model.predict(meta_X_val)
    y_pred_prob_val = meta_model.predict_proba(meta_X_val)[:, 1]

    print("\n📗 VALIDATION METRICS")
    print("Accuracy :", accuracy_score(meta_y_val, y_pred_val))
    print("Precision:", precision_score(meta_y_val, y_pred_val))
    print("Recall   :", recall_score(meta_y_val, y_pred_val))
    print("F1-score :", f1_score(meta_y_val, y_pred_val))
    print("ROC AUC  :", roc_auc_score(meta_y_val, y_pred_prob_val))



Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 368/368 [00:15


Fitting 3 folds for each of 9 candidates, totalling 27 fits


Predicting NN for Dataset 2 (ndarvind/phiusiil-phishing): 100%|█| 46/46 [00:01<0



📘 TRAIN METRICS
Accuracy : 0.9982155754769086
Precision: 0.9971523668639053
Recall   : 0.999740452354468
F1-score : 0.9984447324569524
ROC AUC  : 0.9990979136429019

📗 VALIDATION METRICS
Accuracy : 0.9977482261970515
Precision: 0.9965986394557823
Recall   : 0.9994809047089358
F1-score : 0.9980376911399904
ROC AUC  : 0.9991265884585916


Predicting NN for Dataset 3 (kmack/Phishing_urls): 100%|█| 1032/1032 [00:45<00:0


Fitting 3 folds for each of 9 candidates, totalling 27 fits


Predicting NN for Dataset 3 (kmack/Phishing_urls): 100%|█| 129/129 [00:04<00:00,



📘 TRAIN METRICS
Accuracy : 0.9447033404847973
Precision: 0.9451435983796823
Recall   : 0.9441552241520415
F1-score : 0.9446491527351302
ROC AUC  : 0.9881929120213544

📗 VALIDATION METRICS
Accuracy : 0.9031236744834272
Precision: 0.9022626898178958
Recall   : 0.904095056227456
F1-score : 0.9031779436478977
ROC AUC  : 0.9675889188819361


In [ ]:
#xgboost(nn+xgboost)
from sklearn.linear_model import LogisticRegression

independent_features = ['url_length', 'hostname_length', 'path_length',
    'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
    'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
    'has_subdomain', 'has_suspicious_words', 'domain_length',
    'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
    'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
    'double_slash_in_path']
dependent_features  = 'label'
logistic_model = {}
for name in encoded_data:
    #print(model_dict[name])
    #print(nn_model[name])
    nn_model[name].eval()
    with torch.no_grad():
        model = nn_model[name]
        #print(model)
        x_ = encoded_data[name]['train']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_train = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_train.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_train = torch.cat(nn_preds_train, dim=0).cpu()

    # XGBoost predictions
    xgb_preds_train = model_dict[name].predict_proba(encoded_data[name]['train'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X = np.column_stack((nn_preds_train, xgb_preds_train))
    meta_y = encoded_data[name]['train'][dependent_features]

    with torch.no_grad():
        x_ = encoded_data[name]['valid']['encode']
        x_np = np.stack(x_.values)
        x_ = torch.tensor(x_np, dtype=torch.long).to(device)  # move once to GPU/CPU where model is
        batch_size = 512  # adjust for your GPU memory
        nn_preds_val = []

        # 🔹 Process manually in batches
        for i in tqdm(range(0, len(x_), batch_size), desc=f"Predicting NN for {name}", ncols=80):
            batch_x = x_[i:i + batch_size]
            outputs = model(batch_x)
            nn_preds_val.append(outputs)

    # 🔹 Combine all predictions
    nn_preds_val = torch.cat(nn_preds_val, dim=0).cpu()
    # XGBoost predictions
    xgb_preds_val = model_dict[name].predict_proba(encoded_data[name]['valid'][independent_features])[:, 1]

    # Stack predictions as new features
    meta_X_val = np.column_stack((nn_preds_val, xgb_preds_val))
    meta_y_val = encoded_data[name]['valid'][dependent_features]


    xgb_model = xgb.XGBClassifier(
        n_estimators=500,
        learning_rate=0.01,
        bjective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        max_depth = 10,
        random_state=42,
        
    )

    xgb_model.fit(
        meta_X, meta_y,
        #eval_set=[(meta_X_val, meta_y_val)],
    )


    y_pred_train = meta_model.predict(meta_X)
    y_pred_prob_train = meta_model.predict_proba(meta_X)[:, 1]

    print("\n📘 TRAIN METRICS")
    print("Accuracy :", accuracy_score(meta_y, y_pred_train))
    print("Precision:", precision_score(meta_y, y_pred_train))
    print("Recall   :", recall_score(meta_y, y_pred_train))
    print("F1-score :", f1_score(meta_y, y_pred_train))
    print("ROC AUC  :", roc_auc_score(meta_y, y_pred_prob_train))

    # ===============================
    # 🔹 Validation Metrics
    # ===============================
    y_pred_val = meta_model.predict(meta_X_val)
    y_pred_prob_val = meta_model.predict_proba(meta_X_val)[:, 1]

    print("\n📗 VALIDATION METRICS")
    print("Accuracy :", accuracy_score(meta_y_val, y_pred_val))
    print("Precision:", precision_score(meta_y_val, y_pred_val))
    print("Recall   :", recall_score(meta_y_val, y_pred_val))
    print("F1-score :", f1_score(meta_y_val, y_pred_val))
    print("ROC AUC  :", roc_auc_score(meta_y_val, y_pred_prob_val))


SyntaxError: invalid syntax (1024923085.py, line 16)

# Fedrated learning

In [9]:
#train global model on little part(5%) of dataset
dataset_name = 'Dataset 3 (kmack/Phishing_urls)'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
global_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
loaders = dataloader_dict[dataset_name]
train_loader = loaders["train_loader"]
val_loader = loaders["val_loader"]
lr  = 0.01
print(f"🧩 Using {0.05*100:.0f}% of training data".center(50, '_'))

criterion = nn.BCELoss()
optimizer = torch.optim.NAdam(global_model.parameters(), lr=lr, weight_decay=lr/10)
trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
# Lists to track performance
trainer.train(epochs_list=[3,3,4],frac=0.05,val_frac=0.05, log=2)



c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


___________🧩 Using 5% of training data____________
Epoch 1/3 Training CNN + transformer| Train Loss: 0.6924, Train Acc: 0.5110 | Val Loss: 0.6910, Val Acc: 0.4991
Epoch 2/3 Training CNN + transformer| Train Loss: 0.4956, Train Acc: 0.7867 | Val Loss: 0.4686, Val Acc: 0.7797
Epoch 3/3 Training CNN + transformer| Train Loss: 0.3973, Train Acc: 0.8382 | Val Loss: 0.3686, Val Acc: 0.8401
Epoch 1/3 Training CNN + transformer| Train Loss: 0.3762, Train Acc: 0.8477 | Val Loss: 0.3664, Val Acc: 0.8474
Epoch 2/3 Training CNN + transformer| Train Loss: 0.3523, Train Acc: 0.8564 | Val Loss: 0.3340, Val Acc: 0.8562
Epoch 3/3 Training CNN + transformer| Train Loss: 0.3349, Train Acc: 0.8621 | Val Loss: 0.3080, Val Acc: 0.8701
Epoch 1/4 Training CNN + transformer| Train Loss: 0.3105, Train Acc: 0.8803 | Val Loss: 0.3000, Val Acc: 0.8774
Epoch 2/4 Training CNN + transformer| Train Loss: 0.3010, Train Acc: 0.8855 | Val Loss: 0.2894, Val Acc: 0.8792
Epoch 3/4 Training CNN + transformer| Train Loss: 0.2

In [11]:
import numpy as np
# using alpha to produce unidentical splits high alpha more identical
num_clients = 5
alpha = 0.5
total_data = 0.3
client_fractions = np.random.dirichlet([alpha] * num_clients) * total_data
print(client_fractions)
print(client_fractions.sum())


[0.03499009 0.16437877 0.00607158 0.0008415  0.09371806]
0.3


In [12]:
client_models = []
for i in range(num_clients):
    client_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)                     # fresh instance
    client_model.load_state_dict(global_model.state_dict())
    client_models.append(client_model)

c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [13]:
print('clints initial accuracy')
start = 0
for i, model in enumerate(client_models):
    criterion = nn.BCELoss()
    trainer = Train(model, criterion=criterion, train_loader=train_loader, val_loader=val_loader)
    avg_val_loss, val_acc = trainer.evaluate(frac=client_fractions[i], start=start, log=2)
    start += client_fractions[i]
    print(avg_val_loss, val_acc)

clints initial accuracy
0.27362592497633564 0.8854166666666666
0.27565419590402845 0.8886430678466076
0.304987629254659 0.875
0.3075116127729416 0.875
0.27495867145185027 0.8884391191709845


In [14]:
# trainig clints
start = 0
for i, model in enumerate(client_models):
    criterion = nn.BCELoss()
    trainer = Train(model, criterion=criterion, train_loader=train_loader, val_loader=val_loader)
    trainer.train(epochs_list=[2,0,0],frac=client_fractions[i], val_frac=client_fractions[i], start=start, log=2)
    start += client_fractions[i]


Epoch 1/2 Training CNN + transformer| Train Loss: 0.2941, Train Acc: 0.8810 | Val Loss: 0.2776, Val Acc: 0.8780
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2867, Train Acc: 0.8846 | Val Loss: 0.2750, Val Acc: 0.8885
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2849, Train Acc: 0.8890 | Val Loss: 0.2676, Val Acc: 0.8912
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2779, Train Acc: 0.8867 | Val Loss: 0.2713, Val Acc: 0.8842
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2946, Train Acc: 0.8866 | Val Loss: 0.3125, Val Acc: 0.8646
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2853, Train Acc: 0.8872 | Val Loss: 0.2978, Val Acc: 0.8724
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2533, Train Acc: 0.9085 | Val Loss: 0.3136, Val Acc: 0.8750
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2979, Train Acc: 0.8906 | Val Loss: 0.3009, Val Acc: 0.8750
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2781, Train Acc: 0.8903 | Val Loss: 0.2657, Val Acc:

In [15]:
# tansfering weights and averaging them
import copy
client_weights = [client_model.state_dict() for client_model in client_models]
total_samples = sum(client_fractions)

new_global_state = copy.deepcopy(global_model.state_dict())

# set all params to zero before summing
for key in new_global_state.keys():
    new_global_state[key] = torch.zeros_like(new_global_state[key])

# aggregate client updates
for client_model, n_i in zip(client_models, client_fractions):
    client_state = client_model.state_dict()
    for key in new_global_state.keys():
        new_global_state[key] += client_state[key] * (n_i / total_samples)

# load averaged weights back into global model
global_model.load_state_dict(new_global_state)



<All keys matched successfully>

In [16]:
trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
avg_val_loss, val_acc = trainer.evaluate(frac=0.3, start=0, log=2)
print(avg_val_loss, val_acc)

0.26212733452325887 0.8946197411003236


In [21]:
import copy
def make_clients(num_clients=5, alpha=0.5, total_data=1):
    import numpy as np
    # using alpha to produce unidentical splits high alpha more identical
    client_fractions = np.random.dirichlet([alpha] * num_clients) * total_data
    client_models = []
    for i in range(num_clients):
        client_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)                     # fresh instance
        client_model.load_state_dict(global_model.state_dict())
        client_models.append(client_model)
    return client_model, client_fractions
def train_client(client_models, client_fractions, start=0):
    for i, _ in enumerate(client_models):
        criterion = nn.BCELoss()
        trainer = Train(client_models[i], criterion=criterion, train_loader=train_loader, val_loader=val_loader)
        trainer.train(epochs_list=[2,0,0],frac=client_fractions[i], val_frac=client_fractions[i], start=start, log=2)
        start += client_fractions[i]
def fed_avg(client_models, client_fractions):
    total_samples = sum(client_fractions)
    new_global_state = copy.deepcopy(global_model.state_dict())
    # set all params to zero before summing
    for key in new_global_state.keys():
        new_global_state[key] = torch.zeros_like(new_global_state[key])
    # aggregate client updates
    for client_model, n_i in zip(client_models, client_fractions):
        client_state = client_model.state_dict()
        for key in new_global_state.keys():
            new_global_state[key] += client_state[key] * (n_i / total_samples)
    # load averaged weights back into global model
    global_model.load_state_dict(new_global_state)
def update_clients(client_models):
    for client_model in client_models:
        client_model.load_state_dict(global_model.state_dict())
client_model, client_fractions = make_clients(5, total_data=0.3) 

c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


In [22]:
for i in range(3):
    train_client(client_models, client_fractions, start=0.3)   
    fed_avg(client_models, client_fractions)
    update_clients(client_models)
    trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
    avg_val_loss, val_acc = trainer.evaluate(frac=0.3, start=0, log=2)
    print(f'global model loss {avg_val_loss} accuracy {val_acc}')

Epoch 1/2 Training CNN + transformer| Train Loss: 0.2212, Train Acc: 0.9136 | Val Loss: 0.2286, Val Acc: 0.9040
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2250, Train Acc: 0.9113 | Val Loss: 0.2217, Val Acc: 0.9068
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2292, Train Acc: 0.9146 | Val Loss: 0.2118, Val Acc: 0.9129
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2289, Train Acc: 0.9104 | Val Loss: 0.2184, Val Acc: 0.9089
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2352, Train Acc: 0.9088 | Val Loss: 0.2293, Val Acc: 0.9000
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2282, Train Acc: 0.9116 | Val Loss: 0.2263, Val Acc: 0.9079
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2293, Train Acc: 0.9154 | Val Loss: 0.2071, Val Acc: 0.9190
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2119, Train Acc: 0.9180 | Val Loss: 0.1936, Val Acc: 0.9233
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2304, Train Acc: 0.9079 | Val Loss: 0.2300, Val Acc:

## fedrated learning with persnalization
# meta learing



In [44]:
#train global model on little part(5%) of dataset
dataset_name = 'Dataset 3 (kmack/Phishing_urls)'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
global_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)
loaders = dataloader_dict[dataset_name]
train_loader = loaders["train_loader"]
val_loader = loaders["val_loader"]
lr  = 0.01
print(f"🧩 Using {0.05*100:.0f}% of training data".center(50, '_'))

criterion = nn.BCELoss()
optimizer = torch.optim.NAdam(global_model.parameters(), lr=lr, weight_decay=lr/10)
trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
# Lists to track performance
trainer.train(epochs_list=[3,3,4],frac=0.05,val_frac=0.05, log=2)



___________🧩 Using 5% of training data____________
Epoch 1/3 Training CNN + transformer| Train Loss: 0.5185, Train Acc: 0.7926 | Val Loss: 0.5064, Val Acc: 0.7915
Epoch 2/3 Training CNN + transformer| Train Loss: 0.3922, Train Acc: 0.8453 | Val Loss: 0.3640, Val Acc: 0.8474
Epoch 3/3 Training CNN + transformer| Train Loss: 0.3885, Train Acc: 0.8424 | Val Loss: 0.3510, Val Acc: 0.8579
Epoch 1/3 Training CNN + transformer| Train Loss: 0.3397, Train Acc: 0.8659 | Val Loss: 0.3149, Val Acc: 0.8678
Epoch 2/3 Training CNN + transformer| Train Loss: 0.3321, Train Acc: 0.8696 | Val Loss: 0.3101, Val Acc: 0.8774
Epoch 3/3 Training CNN + transformer| Train Loss: 0.3181, Train Acc: 0.8763 | Val Loss: 0.2937, Val Acc: 0.8786
Epoch 1/4 Training CNN + transformer| Train Loss: 0.3119, Train Acc: 0.8802 | Val Loss: 0.2896, Val Acc: 0.8786
Epoch 2/4 Training CNN + transformer| Train Loss: 0.2980, Train Acc: 0.8829 | Val Loss: 0.2736, Val Acc: 0.8867
Epoch 3/4 Training CNN + transformer| Train Loss: 0.2

In [45]:
import copy
def make_clients(num_clients=5, alpha=0.5, total_data=1):
    import numpy as np
    # using alpha to produce unidentical splits high alpha more identical
    client_fractions = np.random.dirichlet([alpha] * num_clients) * total_data
    client_models = []
    for i in range(num_clients):
        client_model = URLBinaryCNN(vocab_size=len(vocab)).to(device)                     # fresh instance
        client_model.load_state_dict(global_model.state_dict())
        client_models.append(client_model)
    return client_models, client_fractions
def train_client(client_models, client_fractions, start=0):
    for i, _ in enumerate(client_models):
        print("training model: ",i)
        criterion = nn.BCELoss()
        trainer = Train(client_models[i], criterion=criterion, train_loader=train_loader, val_loader=val_loader)
        trainer.train(epochs_list=[2,0,0],frac=client_fractions[i], val_frac=client_fractions[i], start=start, log=2)
        start += client_fractions[i]
def fed_meta_persnalization_learing(global_model, client_models, client_fractions, meta_lr=0.1):
    total_samples = sum(client_fractions)
    weights = [n_i / total_samples for n_i in client_fractions]
    global_state = {
        **global_model.transformer_layer.state_dict(),
        **global_model.shared_layer.state_dict()
    }

    avg_state = copy.deepcopy(global_state)
    for key in avg_state.keys():
        avg_state[key] = torch.zeros_like(avg_state[key])
        for client_model, w in zip(client_models, weights):
            client_state = {
                **client_model.transformer_layer.state_dict(),
                **client_model.shared_layer.state_dict()
            }
            avg_state[key] += client_state[key] * w

    new_state = {}
    for key in global_state.keys():
        new_state[key] = global_state[key] + meta_lr * (avg_state[key] - global_state[key])

    # 5️⃣ Load updated parameters into global model
    global_model.transformer_layer.load_state_dict({
        k: v for k, v in new_state.items() if k in global_model.transformer_layer.state_dict()
    })
    global_model.shared_layer.load_state_dict({
        k: v for k, v in new_state.items() if k in global_model.shared_layer.state_dict()
    })
def update_clients(client_models, global_model):
    for client in client_models:
        client.transformer_layer.load_state_dict(global_model.transformer_layer.state_dict())
        client.shared_layer.load_state_dict(global_model.shared_layer.state_dict())


In [46]:
client_models, client_fractions = make_clients(10, total_data=0.3) 
print(client_fractions)
print('clints initial accuracy')
start = 0.3
for i, model in enumerate(client_models):
    criterion = nn.BCELoss()
    trainer = Train(model, criterion=criterion, train_loader=train_loader, val_loader=val_loader)
    avg_val_loss, val_acc = trainer.evaluate(frac=client_fractions[i], start=start, log=3)
    start += client_fractions[i]
    print(avg_val_loss, val_acc, client_fractions[i])

[5.64915184e-02 1.58020904e-02 4.86587228e-04 9.58656073e-02
 6.45225087e-03 3.59857730e-02 8.68702719e-02 1.46646789e-03
 5.77994956e-04 1.43816393e-06]
clints initial accuracy
starting from batch 618 ending to batch 736 total validation batch size 736
0.29108675960767066 0.8840042372881356 0.05649151835159931
starting from batch 735 ending to batch 769 total validation batch size 769
0.26872584048439474 0.8924632352941176 0.01580209035721166
starting from batch 768 ending to batch 770 total validation batch size 770
0.25756773352622986 0.890625 0.0004865872279650972
starting from batch 769 ending to batch 967 total validation batch size 967
0.27024938583825575 0.8903093434343434 0.0958656072947359
starting from batch 966 ending to batch 981 total validation batch size 981
0.2966575299700101 0.86875 0.006452250873085912
starting from batch 980 ending to batch 1055 total validation batch size 1055
0.28668673038482667 0.8841666666666667 0.03598577298234522
starting from batch 1054 endin

In [ ]:
trainer = Train(global_model, criterion, train_loader=train_loader, val_loader=val_loader)
for i in range(3):
    train_client(client_models, client_fractions, start=0.3)   
    fed_meta_persnalization_learing(global_model, client_models, client_fractions)
    update_clients(client_models, global_model)
    avg_val_loss, val_acc = trainer.evaluate(frac=0.3, start=0, log=2)
    print(f'global model loss {avg_val_loss} accuracy {val_acc}')

training model:  0
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2984, Train Acc: 0.8836 | Val Loss: 0.2839, Val Acc: 0.8803
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2772, Train Acc: 0.8919 | Val Loss: 0.2712, Val Acc: 0.8857
training model:  1
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2935, Train Acc: 0.8830 | Val Loss: 0.3048, Val Acc: 0.8778
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2809, Train Acc: 0.8905 | Val Loss: 0.2796, Val Acc: 0.8883
training model:  2
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2854, Train Acc: 0.9028 | Val Loss: 0.3115, Val Acc: 0.8594
Epoch 2/2 Training CNN + transformer| Train Loss: 0.3740, Train Acc: 0.8576 | Val Loss: 0.3161, Val Acc: 0.8594
training model:  3
Epoch 1/2 Training CNN + transformer| Train Loss: 0.2828, Train Acc: 0.8906 | Val Loss: 0.2650, Val Acc: 0.8903
Epoch 2/2 Training CNN + transformer| Train Loss: 0.2819, Train Acc: 0.8893 | Val Loss: 0.2672, Val Acc: 0.8886
training model:  4
Epoch 1/2

: 